In [4]:
from modelscope import snapshot_download
snapshot_download('AI-ModelScope/bge-large-zh-v1.5', cache_dir='./')


'./AI-ModelScope/bge-large-zh-v1___5'

In [2]:
import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader, CSVLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
from typing import Any, List, Optional
from langchain_community.vectorstores import Chroma
import torch

bge_large_zh_embedding_model_path = "./AI-ModelScope/bge-large-zh-v1___5"
def get_embedding(embedding_model_path):
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}  # set True to compute cosine similarity
    embeddings = HuggingFaceEmbeddings(
        model_name=embedding_model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs,
    )
    return embeddings

# bge_large_zh = get_embedding(bge_large_zh_embedding_model_path)

# loaders_chinese = [
#     # PyMuPDFLoader("./data/24徐涛《核心考案》高清无水印PDF【公众号：薄荷考研】.pdf")
#     PyPDFLoader("./中国近现代史纲要：2023 年版(1).pdf"),
#     PyPDFLoader("《习近平谈治国理政》第一卷.pdf"),
#     PyPDFLoader("《习近平谈治国理政》第二卷.pdf"),
#     PyPDFLoader("《习近平谈治国理政》第三卷.pdf"),
#     PyPDFLoader("《习近平谈治国理政》第四卷.pdf")
#     # 如果需要还可以加入其他文件
# ]
# docs = []
# for loader in loaders_chinese:
#     pages = loader.load()
#     print(len(pages))
#     docs.extend(pages)
# CHUNK_SIZE = 300
# # 知识库中相邻文本重合长度
# OVERLAP_SIZE = 50

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=CHUNK_SIZE,
#     chunk_overlap=OVERLAP_SIZE
# )
# split_docs = text_splitter.split_documents(docs)
# persist_directory = "./vector_db/bge_large_zh"

# vectordb = Chroma.from_documents(
#     documents=split_docs,
#     embedding=bge_large_zh,
#     persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
# )
# vectordb.persist()


In [4]:
import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader, CSVLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
from typing import Any, List, Optional
from langchain_community.vectorstores import Chroma
import torch

bge_large_zh_embedding_model_path = "./AI-ModelScope/bge-large-zh-v1___5"
def get_embedding(embedding_model_path):
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}  # set True to compute cosine similarity
    embeddings = HuggingFaceEmbeddings(
        model_name=embedding_model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs,
    )
    return embeddings

bge_large_zh = get_embedding(bge_large_zh_embedding_model_path)
loader = CSVLoader(file_path="./chioceproblem.csv", encoding="utf-8")
data = loader.load()
# print(data[0])
# split_docs = text_splitter.split_documents(docs)
persist_directory = "./vector_db/choiceproblem"

vectordb = Chroma.from_documents(
    documents=data,
    embedding=bge_large_zh,
    persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
)
vectordb.persist()


page_content='问题: 1、中国共产党领导的多党合作和政治协商制度是一项具有中国特色的(   )。
A: 基本制度
B: 政治制度
C: 社会主义制度
D: 基本政治制度
答案: 基本政治制度' metadata={'source': './chioceproblem.csv', 'row': 0}


/tmp/ipykernel_357/2592206952.py:40: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [3]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

def get_vectordb(file_path: str=None, persist_path: str=None, embedding_path=None):
    """
    返回向量数据库对象
    输入参数：
    question：
    llm:
    vectordb:向量数据库(必要参数),一个对象
    template：提示模版（可选参数）可以自己设计一个提示模版，也有默认使用的
    embedding：可以使用zhipuai等embedding，不输入该参数则默认使用 openai embedding，注意此时api_key不要输错
    """

    embedding = get_embedding(embedding_path)
    if os.path.exists(persist_path):  #持久化目录存在
        contents = os.listdir(persist_path)
        if len(contents) == 0:  #但是下面为空
            #print("目录为空")
            vectordb = create_db(file_path, persist_path, embedding)
        else:
            #print("目录不为空")
            vectordb = load_knowledge_db(persist_path, embedding)
    else: #目录不存在，从头开始创建向量数据库
        vectordb = create_db(file_path, persist_path, embedding)
        #presit_knowledge_db(vectordb)
        # vectordb = load_knowledge_db(persist_path, embedding)

    return vectordb


def create_db(embedding, file_path:str=None, persist_path:str=None):

    loaders_chinese = [
        # PyMuPDFLoader("./data/24徐涛《核心考案》高清无水印PDF【公众号：薄荷考研】.pdf")
        PyPDFLoader("./特色教材.pdf"),
        PyPDFLoader("./中国近现代史纲要：2023 年版(1).pdf")
        PyPDFLoader("《习近平谈治国理政》第一卷")
        PyPDFLoader("《习近平谈治国理政》第二卷")
        PyPDFLoader("《习近平谈治国理政》第三卷")
        PyPDFLoader("《习近平谈治国理政》第四卷")
        # 如果需要还可以加入其他文件
    ]
    docs = []
    for loader in loaders_chinese:
        pages = loader.load()
        print(len(pages))
        docs.extend(pages)
    CHUNK_SIZE = 200
    # 知识库中相邻文本重合长度
    OVERLAP_SIZE = 50
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=OVERLAP_SIZE
    )
    split_docs = text_splitter.split_documents(docs)
    persist_directory = "./vector_db/pdf"

    vectordb = Chroma.from_documents(
        documents=split_docs,
        embedding=embedding,
        persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
    )
    vectordb.persist()
    return vectordb


def load_knowledge_db(persist_path, embedding):
    vectordb = Chroma(
        persist_directory=persist_path,
        embedding_function=embedding
    )
    return vectordb

SyntaxError: invalid syntax. Perhaps you forgot a comma? (4162822435.py, line 39)

In [22]:
from typing import Any, List, Mapping, Optional, Dict
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from zhipuai import ZhipuAI

# 继承自 langchain_core.language_models.llms.LLM,用于调用 ZhipuAI 的语言模型服务
class ZhipuAILLM(LLM):
    # 默认选用 glm-4 模型
    model: str = "glm-4"
    # 温度系数
    temperature: float = 0.1
    # API_Key
    api_key: str = "652a160546149ef4e3ec0ff881beebfe.D3UaKuk7FmiUn9WQ"
    max_tokens: int = 2048

    # 定义 _call 方法：
    # 这个方法实现了实际的 API 调用逻辑：
    # 初始化 ZhipuAI 客户端。
    # 生成请求参数messages。
    # 调用 chat.completions.create 方法获取响应。
    # 返回响应中的内容，如果没有结果则返回错误信息。
    def _call(self, prompt: str, stop: Optional[List[str]] = None,
              run_manager: Optional[CallbackManagerForLLMRun] = None,
              **kwargs: Any):
        # 生成 GLM 模型请求参数的方法：
        # 生成 GLM 模型的请求参数 messages，包括系统消息和用户输入
        def gen_glm_params(prompt):
            '''
            构造 GLM 模型请求参数 messages
            请求参数：
                prompt: 对应的用户提示词
            '''
            messages = [{"role": "user", "content": prompt}]
            return messages

        client = ZhipuAI(
            api_key=self.api_key
        )

        messages = gen_glm_params(prompt)
        response = client.chat.completions.create(
            model=self.model,
            messages=messages,
            temperature=self.temperature,
            max_tokens=self.max_tokens
        )

        if len(response.choices) > 0:
            return response.choices[0].message.content
        return "generate answer error"

    # 定义属性方法：
    # _default_params：返回调用 API 的默认参数。
    # _llm_type：返回模型类型的字符串标识。
    # _identifying_params：返回模型的标识参数。
    # 首先定义一个返回默认参数的方法
    @property
    def _default_params(self) -> Dict[str, Any]:
        """获取调用Ennie API的默认参数。"""
        normal_params = {
            "temperature": self.temperature,
        }
        # print(type(self.model_kwargs))
        return {**normal_params}

    @property
    def _llm_type(self) -> str:
        return "Zhipu"

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {**{"model": self.model}, **self._default_params}


In [66]:
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
# 导入PromptTemplate类，用于创建问题模板
from langchain.chains import RetrievalQA
# 导入RetrievalQA类，用于执行检索问答
import sys
# 导入sys模块，用于操作系统相关功能
from langchain_community.vectorstores import Chroma
from zhipuai import ZhipuAI


sys.path.append("../")
# 添加上级目录到模块搜索路径
# from qa_chain.model_to_llm import model_to_llm
# # 导入model_to_llm函数，用于将模型转换为大语言模型(LLM)
# from qa_chain.get_vectordb import get_vectordb
# # 导入get_vectordb函数，用于获取向量数据库

# 定义 QA_chain_self 类：
# 该类用于创建和管理一个问答链系统，不带历史记录，类的描述文档说明了各个参数的用途。
class QA_chain_self():
    # 类描述
    """
    不带历史记录的问答链
    - model：调用的模型名称
    - temperature：温度系数，控制生成的随机性
    - top_k：返回检索的前k个相似文档
    - file_path：建库文件所在路径
    - persist_path：向量数据库持久化路径
    - api_key：所有模型都需要
    - embeddings：使用的embedding模型
    - embedding_key：使用的embedding模型的秘钥（智谱或者OpenAI）
    - template：可以自定义提示模板，没有输入则使用默认的提示模板default_template_rq
    """

    # 默认的提示模板，用于构建问答的输入
    default_template_rq = """
    使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
    {context}
    问题: {question}
    有用的回答:"""

    template1 = """给你一个任务：给定一段文本，这段文本中第一段是题目，题目中会有括号，后面四行为A，B，C，D四个选项，最后一行是题目的答案。你需要做的是：将题目中的括号以及括号内的内容替换为答案并输出问题，注意不需要输出选项。
    不需要你检查指出题目的对错，不要输出除了以上任务之外的任何回答。
    这是你需要处理的文本：{question}
    """
    template2 = """使用以下背景知识来回答最后的问题。不要试图编造答案。尽量简明扼要地回答。
    背景知识：{context}
    问题：{query}"""


    # 构造函数，初始化类实例
    # 初始化向量数据库和LLM
    # 初始化 PromptTemplate 和 RetrievalQA 类
    def __init__(self, model: str, temperature: float = 0.0, top_k: int = 4, choiceproblem_file_path: str = None,
                 analysis_vectordb_file_path: str = None, persist_path: str = None, api_key: str = None,
                 embedding="zhipuai", embedding_path=None, template=default_template_rq):
        # 类属性初始化
        self.model = model
        self.temperature = temperature
        self.top_k = top_k
        # self.file_path = file_path
        self.persist_path = persist_path
        self.api_key = api_key
        self.embedding = embedding
        # self.embedding_key = embedding_key
        self.embedding_path = embedding_path
        self.template = template
        client = ZhipuAI(api_key="652a160546149ef4e3ec0ff881beebfe.D3UaKuk7FmiUn9WQ")
        # 加载已存在的向量数据库
        self.choiceproblem_vectordb = get_vectordb(choiceproblem_file_path, persist_path + "/choiceproblem", embedding_path)
        self.analysis_vectordb = get_vectordb(analysis_vectordb_file_path, persist_path + "/pdf", embedding_path)
        self.llm = ZhipuAILLM()

        # # 初始化PromptTemplate和RetrievalQA类
        # self.QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"], template=self.template)
        # self.retriever = self.vectordb.as_retriever(search_type="similarity", search_kwargs={'k': self.top_k})
        # self.qa_chain = RetrievalQA.from_chain_type(llm=self.llm, retriever=self.retriever,
        #                                             return_source_documents=True,
        #                                             chain_type_kwargs={"prompt": self.QA_CHAIN_PROMPT})
        self.prompt1 = PromptTemplate(
            input_variables=["question"],
            template=self.template1
        )
        self.chain1 = LLMChain(
            llm=self.llm, prompt=self.prompt1
        )

        self.prompt2 = PromptTemplate(
            input_variables=["context", "query"],
            template=self.template2
        )
        self.chain2 = LLMChain(
            llm=self.llm, prompt=self.prompt2
        )


    # 提供问答功能的方法，根据用户问题调用问答链并返回结果
    def answer(self, query: str = None, temperature=None, top_k=4):
        """
        核心方法，调用问答链
        arguments:
        - question：用户提问
        """
        # 检查问题是否为空
        if len(query) == 0:
            return ""

        # 如果没有指定温度或top_k参数，则使用初始化时的值
        if temperature is None:
            temperature = self.temperature
        if top_k is None:
            top_k = self.top_k

        sim_docs = self.choiceproblem_vectordb.similarity_search(query, k=1)
        question = ""
        for sim_doc in sim_docs:
            print(sim_doc.page_content)
            print("--------------")
            question = question + sim_doc.page_content+'\n'
        response = self.chain1(question)
        print(response)
        print("---------\n")
        context = response['text']

        related_analysis = self.analysis_vectordb.similarity_search(query, k=3)
        for related_doc in related_analysis:
            # print(type(related_doc))
            context = context + related_doc.page_content
        result = self.chain2({'context': context, 'query': query})
        # 调用问答链并返回结果
        # result = self.qa_chain({"query": question, "temperature": temperature, "top_k": top_k})
        return result

In [67]:
prompt: str  # 用户 prompt
model: str = "glm-4-flash"  # 使用的模型
temperature: float = 0.1  # 温度系数
if_history: bool = False  # 是否使用历史对话功能
# API_Key
api_key: str = "652a160546149ef4e3ec0ff881beebfe.D3UaKuk7FmiUn9WQ"
# Secret_Key
secret_key: str = None
# access_token
access_token: str = None
# APPID
appid: str = None
# APISecret
Spark_api_secret: str = None
# Secret_key
Wenxin_secret_key: str = None
# 数据库路径
db_path: str = "./vector_db"
# 源文件路径
file_path: str = "./中国近现代史纲要：2023 年版(1).pdf"
# prompt template
prompt_template: str = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
有用的回答:"""
# Template 变量
input_variables: list = ["context", "question"]
# Embdding
embedding: str = "m3e"
# Top K
top_k: int = 5
# embedding_key
embedding_key = "652a160546149ef4e3ec0ff881beebfe.D3UaKuk7FmiUn9WQ"

choiceproblem_file_path="./vector_db/choiceproblem"
analysis_vectordb_file_path="./vector_db/pdf"
embedding_path = "./AI-ModelScope/bge-large-zh-v1___5"

chain = QA_chain_self(model=model, temperature=temperature, top_k=top_k,
                              choiceproblem_file_path=choiceproblem_file_path, analysis_vectordb_file_path=analysis_vectordb_file_path,persist_path=db_path,
                              api_key=api_key, embedding_path=embedding_path, template=prompt_template)

response = chain.answer(query="大革命失败后，点燃工农武装割据星星之火是指什么")
print(response)

问题: 379、大革命失败后，点燃"工农武装割据""星星之火"是指 ( )
A: 中华苏维埃共和国成立
B: 南昌起义
C: "八七"会议
D: 井冈山根据地建立
答案: 井冈山根据地建立
--------------
{'question': '问题: 379、大革命失败后，点燃"工农武装割据""星星之火"是指 ( )\nA: 中华苏维埃共和国成立\nB: 南昌起义\nC: "八七"会议\nD: 井冈山根据地建立\n答案: 井冈山根据地建立\n', 'text': '大革命失败后，点燃"工农武装割据""星星之火"是指井冈山根据地建立。'}
---------

{'context': '大革命失败后，点燃"工农武装割据""星星之火"是指井冈山根据地建立。i j 1 1「 11 1 I L丿,)i<i, I,\'\';\', I r J 1 i, \n《井冈山的斗争》曲篇文心，明确指出以农＼ lV为主耍经济的中\n国革命，以军事发展暴动，是一种持征；深亥lj论证［红色政\n权能够长期存在并发展的主客观条件，提出了工农武装割据\n的思想。 1930年1月，毛泽东在《星星之火，可以僚原》一\n文中进一步指出：红军、游击队和红色区域的建立和发展，l 27 \n谬\n东：《星起着先锋作用和表率作用。由共产党直接领导的第四军独立团，是一个突\n出的例证。独立团在北伐中战功卓著，使第四军赢得了“铁军＇的称号。此\n外，共产党人还建立了一定数釐的工农武装（工人纠察队、农民自卫军等），\n配合正规军作战，而上海工人的起义武装更是充当了解放上海的主力。\n一：＾、大，节命的失败及其教训\n丸店命 I_1一向失败 在大革命初期和中期，中国共产党的路线基本是正确日着草鞋十革命，夜走山路访贫农”等民歌广泛传唱。\n革命根据地生机勃勃的景象，同国民党统治区民不聊生的悲惨景象形\n成鲜明对照。根据地成为新民主主义共和国的雏形，使身陷苦难深渊的中\n国人民看到了光明和希望。\nL地，中命战争的四巾挫仇 中国革命的复兴和发展并不是一帆风顺\n的。从 1927年7月大革命失败到 1935年1月遵义会议召开前，“左”倾', 'query': '大革命失败后，点燃工农武装割据星星之火是指什么', 'text': '大革命失败后，点燃“工农武装割据星星之火”是指井冈山根据地的建立和工农武装割据思想的实践。

In [ ]:
problems = ["近代中国的社会性质是什么？",
           "近代中国的主要矛盾是什么？",
           "如何认识近代中国社会的落伍",
           "太平天国运动的历史意义与局限",
           "洋务运动的历史意义与局限",
           "辛亥革命的纲领，意义是什么？",
           "新文化运动的内容是",
           "毛泽东思想形成的标志是什么？",
           "中国共产党史上生死攸关的转折点是什么？",
           "中国抗日战争在世界反法西斯战争中的地位和作用？",
           "中国共产党领导的多党合作，政治协商的格局是怎样形成的？",
            "中国革命胜利的原因和基本经验有哪些？",
            "新民主主义社会的性质",
            "中国实行社会主义改造的国内外条件？",
            "新冠肺炎疫情发生后，党中央将疫情防控作为头等大事来抓，习近平总书记亲自指挥、亲自部署，坚持把什么放在第一位？",
            "深化供给侧结构性改革，必须持续推进(   )，优化市场供求结构",
            "近代以来中华民族最伟大的梦想是什么？",
            "习近平总书记强调，我们党全面领导、长期执政，面临的最大挑战是什么？",
            "习近平强军思想是什么",
            "党的十九届四中全会指出，党和人民在长期实践探索中形成的科学制度体系是指",
            "2020年脱贫攻坚任务完成后，我国将有1亿左右贫困人口实现脱贫，这也意味着我国将提前（）实现联合国2030年可持续发展议程的减贫目标",
            "党的十九大提出：从2035年到本世纪中叶，在基本实现现代化的基础上，再奋斗15年，把我国建成（）的社会主义现代化强国",
            "新时代党的建设总要求是坚持和加强党的全面领导，坚持党要管党、全面从严治党，以(   )为主线",
            "习近平总书记在参加内蒙古代表团的审议时发表重要讲话指出，我们党没有自己特殊的利益，党在任何时候都把（）放在第一位",
            "2014年十八届四中全会通过了(   )，明确提出全面推进依法治国，加快建设法治中国，开启了中国特色社会主义法治道路的新征程",
            "中国共产党 ( ) 将毛泽东思想规定为中国共产党的一切工作的指针"
          ]
for problem in problems:
    res = vectordb.similarity_search_with_relevance_scores(query=problem, k=1)
    print(res)
    print("-----------")


In [12]:
snapshot_download('iic/nlp_gte_sentence-embedding_chinese-large', cache_dir='./')

2024-09-23 13:59:58,811 - modelscope - WARNING - Model revision not specified, use revision: v1.1.0


'./iic/nlp_gte_sentence-embedding_chinese-large'

In [16]:
nlp_gte_embedding_model_path = "./iic/nlp_gte_sentence-embedding_chinese-large"

nlp_gte = get_embedding(nlp_gte_embedding_model_path)

loaders_chinese = [
    # PyMuPDFLoader("./data/24徐涛《核心考案》高清无水印PDF【公众号：薄荷考研】.pdf")
        PyPDFLoader("./中国近现代史纲要：2023 年版(1).pdf"),
        PyPDFLoader("《习近平谈治国理政》第一卷.pdf"),
        PyPDFLoader("《习近平谈治国理政》第二卷.pdf"),
        PyPDFLoader("《习近平谈治国理政》第三卷.pdf"),
        PyPDFLoader("《习近平谈治国理政》第四卷.pdf")
    # 如果需要还可以加入其他文件
]
docs = []
for loader in loaders_chinese:
    docs.extend(loader.load())
CHUNK_SIZE = 300
# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)
split_docs = text_splitter.split_documents(docs)
persist_directory = "./vector_db/nlp__gte"

vectordb = Chroma.from_documents(
    documents=split_docs,
    embedding=nlp_gte,
    persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
)
vectordb.persist()

No sentence-transformers model found with name ./iic/nlp_gte_sentence-embedding_chinese-large. Creating a new one with mean pooling.


In [23]:
nlp_gte_embedding_model_path = "./iic/nlp_gte_sentence-embedding_chinese-large"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True} 
nlp_gte=embeddings=HuggingFaceEmbeddings(
        model_name=nlp_gte_embedding_model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs,
    )
persist_directory = "./vector_db/nlp__gte"
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=nlp_gte
) 

problems = ["近代中国的社会性质是什么？",
           "近代中国的主要矛盾是什么？",
           "如何认识近代中国社会的落伍",
           "太平天国运动的历史意义与局限",
           "洋务运动的历史意义与局限",
           "辛亥革命的纲领，意义是什么？",
           "新文化运动的内容是",
           "毛泽东思想形成的标志是什么？",
           "中国共产党史上生死攸关的转折点是什么？",
           "中国抗日战争在世界反法西斯战争中的地位和作用？",
           "中国共产党领导的多党合作，政治协商的格局是怎样形成的？",
            "中国革命胜利的原因和基本经验有哪些？",
            "新民主主义社会的性质",
            "中国实行社会主义改造的国内外条件？",
            "新冠肺炎疫情发生后，党中央将疫情防控作为头等大事来抓，习近平总书记亲自指挥、亲自部署，坚持把什么放在第一位？",
            "深化供给侧结构性改革，必须持续推进(   )，优化市场供求结构",
            "近代以来中华民族最伟大的梦想是什么？",
            "习近平总书记强调，我们党全面领导、长期执政，面临的最大挑战是什么？",
            "习近平强军思想是什么",
            "党的十九届四中全会指出，党和人民在长期实践探索中形成的科学制度体系是指",
            "2020年脱贫攻坚任务完成后，我国将有1亿左右贫困人口实现脱贫，这也意味着我国将提前（）实现联合国2030年可持续发展议程的减贫目标",
            "党的十九大提出：从2035年到本世纪中叶，在基本实现现代化的基础上，再奋斗15年，把我国建成（）的社会主义现代化强国",
            "新时代党的建设总要求是坚持和加强党的全面领导，坚持党要管党、全面从严治党，以(   )为主线",
            "习近平总书记在参加内蒙古代表团的审议时发表重要讲话指出，我们党没有自己特殊的利益，党在任何时候都把（）放在第一位",
            "2014年十八届四中全会通过了(   )，明确提出全面推进依法治国，加快建设法治中国，开启了中国特色社会主义法治道路的新征程",
            "中国共产党 ( ) 将毛泽东思想规定为中国共产党的一切工作的指针"
          ]
for problem in problems:
    res = vectordb.similarity_search_with_relevance_scores(query=problem, k=1)
    print(res)
    print("-----------")

No sentence-transformers model found with name ./iic/nlp_gte_sentence-embedding_chinese-large. Creating a new one with mean pooling.


[(Document(metadata={'page': 15, 'source': './中国近现代史纲要：2023 年版(1).pdf'}, page_content='半殖民地半封建社会，是近代以来中国在外国资本－帝国主义。势力入侵\n及其与中国封建主义势力相结合条件下，逐步形成的一种从屈于资本主义\n世界体系的畸形社会形态。从鸦片战争开始，到 1949年中华人民共和国\n成立前，中国都属于半殖民地半封建补会。\n半殖民地半封建社会性质，决定了近代中国社会矛盾呈现错综复杂的\n状况。在诸多籵会矛盾中，主要矛盾是帝国主义和中华民族的矛盾、封建\n主义和人民大众的矛盾。上述主要矛盾贯穿中国半殖民地半封建社会的始\n. ．一心~··· ` -- -”“ \n@ 资本－帝国主义，涵盖自由竞争阶段的资本主义和垄断阶段的资本主义即\n帝国主义。'), 0.4169695790257645)]
-----------
[(Document(metadata={'page': 53, 'source': '《习近平谈治国理政》第四卷.pdf'}, page_content='当注意的。”近代以后，中国社会矛盾错综复杂，有帝国主义和\n中华民族的矛盾、封建主义和人民大众的矛盾，有资产阶级和无\n产阶级的矛盾、反动统治阶级内部的矛盾，而帝国主义和中华民'), 0.5949905757309886)]
-----------
[(Document(metadata={'page': 53, 'source': '《习近平谈治国理政》第四卷.pdf'}, page_content='当注意的。”近代以后，中国社会矛盾错综复杂，有帝国主义和\n中华民族的矛盾、封建主义和人民大众的矛盾，有资产阶级和无\n产阶级的矛盾、反动统治阶级内部的矛盾，而帝国主义和中华民'), 0.4518865149962513)]
-----------
[(Document(metadata={'page': 65, 'source': './中国近现代史纲要：2023 年版(1).pdf'}, page_content="俨\n1h,I,11,i`Ap`_I'1,\n＇，＇\n,I,_11\n失败。\na心今、农民斗争的总义和凡限\n人平人 1l i衣l心店义的丿力史义\n太平天国起义虽然失败了，但它具有"

In [11]:
snapshot_download('Xorbits/bge-m3', cache_dir='./')

2024-09-23 13:58:45,742 - modelscope - WARNING - Model revision not specified, use revision: v0.0.1


'./Xorbits/bge-m3'

In [24]:
bge_m3_embedding_model_path = "./Xorbits/bge-m3"


bge_m3 = get_embedding(bge_m3_embedding_model_path)

loaders_chinese = [
    # PyMuPDFLoader("./data/24徐涛《核心考案》高清无水印PDF【公众号：薄荷考研】.pdf")
        PyPDFLoader("./中国近现代史纲要：2023 年版(1).pdf"),
        PyPDFLoader("./《习近平谈治国理政》第一卷.pdf"),
        PyPDFLoader("./《习近平谈治国理政》第二卷.pdf"),
        PyPDFLoader("./《习近平谈治国理政》第三卷.pdf"),
        PyPDFLoader("./《习近平谈治国理政》第四卷.pdf")
    # 如果需要还可以加入其他文件
]
docs = []
for loader in loaders_chinese:
    docs.extend(loader.load())
CHUNK_SIZE = 300
# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)
split_docs = text_splitter.split_documents(docs)
persist_directory = "./vector_db/bge__m3"

vectordb = Chroma.from_documents(
    documents=split_docs,
    embedding=bge_m3,
    persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
)
vectordb.persist()

In [29]:
problems = ["近代中国的社会性质是什么？",
           "近代中国的主要矛盾是什么？",
           "如何认识近代中国社会的落伍",
           "太平天国运动的历史意义与局限",
           "洋务运动的历史意义与局限",
           "辛亥革命的纲领，意义是什么？",
           "新文化运动的内容是",
           "毛泽东思想形成的标志是什么？",
           "中国共产党史上生死攸关的转折点是什么？",
           "中国抗日战争在世界反法西斯战争中的地位和作用？",
           "中国共产党领导的多党合作，政治协商的格局是怎样形成的？",
            "中国革命胜利的原因和基本经验有哪些？",
            "新民主主义社会的性质",
            "中国实行社会主义改造的国内外条件？",
            "新冠肺炎疫情发生后，党中央将疫情防控作为头等大事来抓，习近平总书记亲自指挥、亲自部署，坚持把什么放在第一位？",
            "深化供给侧结构性改革，必须持续推进(   )，优化市场供求结构",
            "近代以来中华民族最伟大的梦想是什么？",
            "习近平总书记强调，我们党全面领导、长期执政，面临的最大挑战是什么？",
            "习近平强军思想是什么",
            "党的十九届四中全会指出，党和人民在长期实践探索中形成的科学制度体系是指",
            "2020年脱贫攻坚任务完成后，我国将有1亿左右贫困人口实现脱贫，这也意味着我国将提前（）实现联合国2030年可持续发展议程的减贫目标",
            "党的十九大提出：从2035年到本世纪中叶，在基本实现现代化的基础上，再奋斗15年，把我国建成（）的社会主义现代化强国",
            "新时代党的建设总要求是坚持和加强党的全面领导，坚持党要管党、全面从严治党，以(   )为主线",
            "习近平总书记在参加内蒙古代表团的审议时发表重要讲话指出，我们党没有自己特殊的利益，党在任何时候都把（）放在第一位",
            "2014年十八届四中全会通过了(   )，明确提出全面推进依法治国，加快建设法治中国，开启了中国特色社会主义法治道路的新征程",
            "中国共产党 ( ) 将毛泽东思想规定为中国共产党的一切工作的指针"
          ]
for problem in problems:
    res = vectordb.similarity_search_with_relevance_scores(query=problem, k=1)
    print(res)
    print("-----------")

[(Document(metadata={'page': 33, 'source': './中国近现代史纲要：2023 年版(1).pdf'}, page_content='从近代中国的历史进程，可以看到中国半殖民地半封建社会有以下一\n些基本特征：\n第一，资本－帝国主义侵略势力不但逐步操纵了中国的财政和经济命\n脉，而且逐步控制了中国的政治，日益成为支配中国的决定性力狱。\n第二，中国的封建势力日益衰败并同外国侵略势力勾结，成为资本－\n帝国主义压迫、奴役中国人民的社会基础和统治支柱。\n第三，中国自然经济的基础虽然遭到破坏，但是封建剥削制度的根基\n即封建地主的土地所有制依然在广大地区内保持着，成为中国发展进步的\n严重障碍。\n第四，中国新兴的民族资本主义经济虽然已经产生，并在政治、文化\n生活中起了一定的作用，但是在帝国主义和封建主义的压迫下，它的发展'), 0.49691469065874405)]
-----------
[(Document(metadata={'page': 53, 'source': '《习近平谈治国理政》第四卷.pdf'}, page_content='当注意的。”近代以后，中国社会矛盾错综复杂，有帝国主义和\n中华民族的矛盾、封建主义和人民大众的矛盾，有资产阶级和无\n产阶级的矛盾、反动统治阶级内部的矛盾，而帝国主义和中华民'), 0.5714230274212123)]
-----------
[(Document(metadata={'page': 53, 'source': '《习近平谈治国理政》第四卷.pdf'}, page_content='当注意的。”近代以后，中国社会矛盾错综复杂，有帝国主义和\n中华民族的矛盾、封建主义和人民大众的矛盾，有资产阶级和无\n产阶级的矛盾、反动统治阶级内部的矛盾，而帝国主义和中华民'), 0.4354551024889456)]
-----------
[(Document(metadata={'page': 66, 'source': './中国近现代史纲要：2023 年版(1).pdf'}, page_content='弟．， I，J}.＼「人 1h埣动的起，许53 \n不可磨灭的历史功绩和重大的历史意义。\n太平天国起义沉重打击了封建统治阶级，强烈撼动了清政府的统治根\n基。这

In [30]:
snapshot_download('Jerry0/M3E-large', cache_dir='./')

'./Jerry0/M3E-large'

In [ ]:
M3E_large_embedding_model_path = "./Jerry0/M3E-large"


M3E_large = get_embedding(M3E_large_embedding_model_path)

loaders_chinese = [
    # PyMuPDFLoader("./data/24徐涛《核心考案》高清无水印PDF【公众号：薄荷考研】.pdf")
        PyPDFLoader("./中国近现代史纲要：2023 年版(1).pdf"),
        PyPDFLoader("./《习近平谈治国理政》第一卷.pdf"),
        PyPDFLoader("./《习近平谈治国理政》第二卷.pdf"),
        PyPDFLoader("./《习近平谈治国理政》第三卷.pdf"),
        PyPDFLoader("./《习近平谈治国理政》第四卷.pdf")
    # 如果需要还可以加入其他文件
]
docs = []
for loader in loaders_chinese:
    docs.extend(loader.load())
CHUNK_SIZE = 300
# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)
split_docs = text_splitter.split_documents(docs)
persist_directory = "./vector_db/M3E_large"

vectordb = Chroma.from_documents(
    documents=split_docs,
    embedding=M3E_large,
    persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
)
vectordb.persist()

No sentence-transformers model found with name ./Jerry0/M3E-large. Creating a new one with mean pooling.


In [32]:
problems = ["近代中国的社会性质是什么？",
           "近代中国的主要矛盾是什么？",
           "如何认识近代中国社会的落伍",
           "太平天国运动的历史意义与局限",
           "洋务运动的历史意义与局限",
           "辛亥革命的纲领，意义是什么？",
           "新文化运动的内容是",
           "毛泽东思想形成的标志是什么？",
           "中国共产党史上生死攸关的转折点是什么？",
           "中国抗日战争在世界反法西斯战争中的地位和作用？",
           "中国共产党领导的多党合作，政治协商的格局是怎样形成的？",
            "中国革命胜利的原因和基本经验有哪些？",
            "新民主主义社会的性质",
            "中国实行社会主义改造的国内外条件？",
            "新冠肺炎疫情发生后，党中央将疫情防控作为头等大事来抓，习近平总书记亲自指挥、亲自部署，坚持把什么放在第一位？",
            "深化供给侧结构性改革，必须持续推进(   )，优化市场供求结构",
            "近代以来中华民族最伟大的梦想是什么？",
            "习近平总书记强调，我们党全面领导、长期执政，面临的最大挑战是什么？",
            "习近平强军思想是什么",
            "党的十九届四中全会指出，党和人民在长期实践探索中形成的科学制度体系是指",
            "2020年脱贫攻坚任务完成后，我国将有1亿左右贫困人口实现脱贫，这也意味着我国将提前（）实现联合国2030年可持续发展议程的减贫目标",
            "党的十九大提出：从2035年到本世纪中叶，在基本实现现代化的基础上，再奋斗15年，把我国建成（）的社会主义现代化强国",
            "新时代党的建设总要求是坚持和加强党的全面领导，坚持党要管党、全面从严治党，以(   )为主线",
            "习近平总书记在参加内蒙古代表团的审议时发表重要讲话指出，我们党没有自己特殊的利益，党在任何时候都把（）放在第一位",
            "2014年十八届四中全会通过了(   )，明确提出全面推进依法治国，加快建设法治中国，开启了中国特色社会主义法治道路的新征程",
            "中国共产党 ( ) 将毛泽东思想规定为中国共产党的一切工作的指针"
          ]
for problem in problems:
    res = vectordb.similarity_search_with_relevance_scores(query=problem, k=1)
    print(res)
    print("-----------")

[(Document(metadata={'page': 33, 'source': './中国近现代史纲要：2023 年版(1).pdf'}, page_content='从近代中国的历史进程，可以看到中国半殖民地半封建社会有以下一\n些基本特征：\n第一，资本－帝国主义侵略势力不但逐步操纵了中国的财政和经济命\n脉，而且逐步控制了中国的政治，日益成为支配中国的决定性力狱。\n第二，中国的封建势力日益衰败并同外国侵略势力勾结，成为资本－\n帝国主义压迫、奴役中国人民的社会基础和统治支柱。\n第三，中国自然经济的基础虽然遭到破坏，但是封建剥削制度的根基\n即封建地主的土地所有制依然在广大地区内保持着，成为中国发展进步的\n严重障碍。\n第四，中国新兴的民族资本主义经济虽然已经产生，并在政治、文化\n生活中起了一定的作用，但是在帝国主义和封建主义的压迫下，它的发展'), 0.502993994234804)]
-----------
[(Document(metadata={'page': 35, 'source': './中国近现代史纲要：2023 年版(1).pdf'}, page_content='产阶级勹地主阶级的矛盾，无产阶级与资产阶级的矛盾，封建统治阶级内\n部各梊团派系的矛盾，各帝国主义国家在中国争夺的矛盾，等等。在这些\n社会矛盾中，古支厅地位的卞要矛盾，是帝国主义和中华民族的矛盾，封\n建主义和人民大众的矛盾。这两对卞要矛盾及其斗争贯穿整个中国半殖民\n地平封注补会的妎终，并对中国近代社会的发展变化起着决定性的作用。\n中同沂代社会的两对主要矛盾是＿勺相交织在一起的，而帝国主义和中\n华民族的矛盾，处最主要的矛盾。一般米说，当资本—帝国主义向中国发\n动侵略战争时，中匡内部各阶级，除一些叛国分子外，能够暂时地团结起\n来举行民族战争去反对外国侵略。这时，民族矛盾特别尖锐，阶级矛盾暂'), 0.4788615938612847)]
-----------
[(Document(metadata={'page': 235, 'source': './《习近平谈治国理政》第二卷.pdf'}, page_content='入，如上海的“十里洋场”、天津的工业、武汉的军工生产也曾名震\n一时，但总体上国家是贫穷落后、战乱不已的，在时代前进潮流中\n掉队了。这一

In [33]:
snapshot_download('AI-ModelScope/tao-8k', cache_dir='./')

'./AI-ModelScope/tao-8k'

In [3]:
tao_8k_embedding_model_path = "./trained_tao1"


tao_8k = get_embedding(tao_8k_embedding_model_path)

loaders_chinese = [
        PyPDFLoader("./中国近现代史纲要：2023 年版(1).pdf"),
        PyPDFLoader("./《习近平谈治国理政》第一卷.pdf"),
        PyPDFLoader("./《习近平谈治国理政》第二卷.pdf"),
        PyPDFLoader("./《习近平谈治国理政》第三卷.pdf"),
        PyPDFLoader("./《习近平谈治国理政》第四卷.pdf")
    # 如果需要还可以加入其他文件
]
docs = []
for loader in loaders_chinese:
    pages = loader.load()
    print(len(pages))
    docs.extend(pages)
CHUNK_SIZE = 300
# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)
split_docs = text_splitter.split_documents(docs)
persist_directory = "./vector_db/trained_tao1"

vectordb = Chroma.from_documents(
    documents=split_docs,
    embedding=tao_8k,
    persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
)
vectordb.persist()

429
536
542
427
538


/tmp/ipykernel_3951/4270446793.py:35: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [4]:
problems = ["近代中国的社会性质是什么？",
           "近代中国的主要矛盾是什么？",
           "如何认识近代中国社会的落伍",
           "太平天国运动的历史意义与局限",
           "洋务运动的历史意义与局限",
           "辛亥革命的纲领，意义是什么？",
           "新文化运动的内容是",
           "毛泽东思想形成的标志是什么？",
           "中国共产党史上生死攸关的转折点是什么？",
           "中国抗日战争在世界反法西斯战争中的地位和作用？",
           "中国共产党领导的多党合作，政治协商的格局是怎样形成的？",
            "中国革命胜利的原因和基本经验有哪些？",
            "新民主主义社会的性质",
            "中国实行社会主义改造的国内外条件？",
            "新冠肺炎疫情发生后，党中央将疫情防控作为头等大事来抓，习近平总书记亲自指挥、亲自部署，坚持把什么放在第一位？",
            "深化供给侧结构性改革，必须持续推进(   )，优化市场供求结构",
            "近代以来中华民族最伟大的梦想是什么？",
            "习近平总书记强调，我们党全面领导、长期执政，面临的最大挑战是什么？",
            "习近平强军思想是什么",
            "党的十九届四中全会指出，党和人民在长期实践探索中形成的科学制度体系是指",
            "2020年脱贫攻坚任务完成后，我国将有1亿左右贫困人口实现脱贫，这也意味着我国将提前（）实现联合国2030年可持续发展议程的减贫目标",
            "党的十九大提出：从2035年到本世纪中叶，在基本实现现代化的基础上，再奋斗15年，把我国建成（）的社会主义现代化强国",
            "新时代党的建设总要求是坚持和加强党的全面领导，坚持党要管党、全面从严治党，以(   )为主线",
            "习近平总书记在参加内蒙古代表团的审议时发表重要讲话指出，我们党没有自己特殊的利益，党在任何时候都把（）放在第一位",
            "2014年十八届四中全会通过了(   )，明确提出全面推进依法治国，加快建设法治中国，开启了中国特色社会主义法治道路的新征程",
            "中国共产党 ( ) 将毛泽东思想规定为中国共产党的一切工作的指针"
          ]
for problem in problems:
    res = vectordb.similarity_search_with_relevance_scores(query=problem, k=1)
    print(res)
    print("-----------")

[(Document(metadata={'page': 33, 'source': './中国近现代史纲要：2023 年版(1).pdf'}, page_content='从近代中国的历史进程，可以看到中国半殖民地半封建社会有以下一\n些基本特征：\n第一，资本－帝国主义侵略势力不但逐步操纵了中国的财政和经济命\n脉，而且逐步控制了中国的政治，日益成为支配中国的决定性力狱。\n第二，中国的封建势力日益衰败并同外国侵略势力勾结，成为资本－\n帝国主义压迫、奴役中国人民的社会基础和统治支柱。\n第三，中国自然经济的基础虽然遭到破坏，但是封建剥削制度的根基\n即封建地主的土地所有制依然在广大地区内保持着，成为中国发展进步的\n严重障碍。\n第四，中国新兴的民族资本主义经济虽然已经产生，并在政治、文化\n生活中起了一定的作用，但是在帝国主义和封建主义的压迫下，它的发展'), 0.16065242398870117)]
-----------
[(Document(metadata={'page': 53, 'source': './《习近平谈治国理政》第四卷.pdf'}, page_content='当注意的。”近代以后，中国社会矛盾错综复杂，有帝国主义和\n中华民族的矛盾、封建主义和人民大众的矛盾，有资产阶级和无\n产阶级的矛盾、反动统治阶级内部的矛盾，而帝国主义和中华民'), 0.4948164519524777)]
-----------
[(Document(metadata={'page': 199, 'source': './《习近平谈治国理政》第三卷.pdf'}, page_content='思维、零和博弈愈发陈旧落伍，妄自尊大或独善其身只能四处碰壁。只有坚持和平发展、\n携手合作，才能真正实现共赢、多赢。\n当今世界，开放融通的潮流滚滚向前。人类社会发展的历史告诉我们，开放带来进步，\n封闭必然落后。世界已经成为你中有我、我中有你的地球村，各国经济社会发展日益相互\n联系、相互影响，推进互联互通、加快融合发展成为促进共同繁荣发展的必然选择。\n当今世界，变革创新的潮流滚滚向前。中国的先人们早在2500多年前就认识到：“苟\n利于民，不必法古；苟周于事，不必循俗”。变革创新是推动人类社会向前发展的根本动\n力。谁排斥变革，谁拒绝创新，谁就会落后于时代，谁就会

/tmp/ipykernel_3951/2830832776.py:29: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'page': 199, 'source': './《习近平谈治国理政》第三卷.pdf'}, page_content='思维、零和博弈愈发陈旧落伍，妄自尊大或独善其身只能四处碰壁。只有坚持和平发展、\n携手合作，才能真正实现共赢、多赢。\n当今世界，开放融通的潮流滚滚向前。人类社会发展的历史告诉我们，开放带来进步，\n封闭必然落后。世界已经成为你中有我、我中有你的地球村，各国经济社会发展日益相互\n联系、相互影响，推进互联互通、加快融合发展成为促进共同繁荣发展的必然选择。\n当今世界，变革创新的潮流滚滚向前。中国的先人们早在2500多年前就认识到：“苟\n利于民，不必法古；苟周于事，不必循俗”。变革创新是推动人类社会向前发展的根本动\n力。谁排斥变革，谁拒绝创新，谁就会落后于时代，谁就会被历史淘汰。'), -0.03894434525937118)]
  res = vectordb.similarity_search_with_relevance_scores(query=problem, k=1)


[(Document(metadata={'page': 227, 'source': './《习近平谈治国理政》第四卷.pdf'}, page_content='的关系，在性质上要区分，在定位上要明确，规范和引导各类资\n本健康发展。\n要正确处理资本和利益分配问题。我国社会主义的国家性质\n决定了我们必须坚持按劳分配为主体、多种分配方式并存，在社'), 0.06137636525826051)]
-----------
[(Document(metadata={'page': 136, 'source': './《习近平谈治国理政》第四卷.pdf'}, page_content='从国内看，我国继续发展具有多方面优势和条件，也面临不\n少困难和挑战。关键是要用全面、辩证、长远的眼光看问题，积\n极拓展发展新空间。\n第一，深刻认识社会主要矛盾变化，增强解决发展不平衡不\n充分问题的系统性。当前，我国发展面临的主要问题是创新能力\n不适应高质量发展要求，农业基础还不稳固，城乡区域发展和收\n入分配差距较大，生态环保任重道远，民生保障存在短板，社会\n治理还有弱项。归结起来，就是发展不平衡发展不充分。发展不\n平衡，主要是各区域各领域各方面存在失衡现象，制约了整体发\n展水平提升；发展不充分，主要是我国全面实现社会主义现代化\n还有相当长的路要走，发展任务仍然很重。推动解决这些问题，'), 0.07858054023301342)]
-----------
[(Document(metadata={'page': 378, 'source': './中国近现代史纲要：2023 年版(1).pdf'}, page_content='发生的最严重的传染病大流行。\n新冠肺炎疫清发生后，党中央将疫情防控作为头等大事来抓。习近平\n亲自指挥、亲自部署，坚持把人民生命安全和身体健康放在第一位，提出\n坚定信心、同舟共济、科学防治、精准施策的总要求。从 2020年大年初一\n起，习近平先后主持召开 14次中央政治局常委会会议、 4次中央政治局会\n议以及多次党的重要会议，敏锐洞察、果敢决策，科学指引、沉着应对，\n周密部署武汉保卫战、湖北保卫战，因时因势制定重大战略策略，带领全\n党全军全国各族人民迅速打响疫情防控的人民战争、总体战、阻击战。\n在党中央坚强领导下，中国人民风雨

In [36]:
snapshot_download('Jerry0/text2vec-large-chinese', cache_dir='./')

'./Jerry0/text2vec-large-chinese'

In [38]:
text2vec_embedding_model_path = "./Jerry0/text2vec-large-chinese"

text2vec = get_embedding(text2vec_embedding_model_path)

loaders_chinese = [
    # PyMuPDFLoader("./data/24徐涛《核心考案》高清无水印PDF【公众号：薄荷考研】.pdf")
        PyPDFLoader("./中国近现代史纲要：2023 年版(1).pdf"),
        PyPDFLoader("./《习近平谈治国理政》第一卷.pdf"),
        PyPDFLoader("./《习近平谈治国理政》第二卷.pdf"),
        PyPDFLoader("./《习近平谈治国理政》第三卷.pdf"),
        PyPDFLoader("./《习近平谈治国理政》第四卷.pdf")
    # 如果需要还可以加入其他文件
]
docs = []
for loader in loaders_chinese:
    pages = loader.load()
    print(len(pages))
    docs.extend(pages)
CHUNK_SIZE = 300
# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)
split_docs = text_splitter.split_documents(docs)
persist_directory = "./vector_db/text2vec"

vectordb = Chroma.from_documents(
    documents=split_docs,
    embedding=text2vec,
    persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
)
vectordb.persist()

No sentence-transformers model found with name ./Jerry0/text2vec-large-chinese. Creating a new one with mean pooling.


429
536
542
427
538


In [39]:
problems = ["近代中国的社会性质是什么？",
           "近代中国的主要矛盾是什么？",
           "如何认识近代中国社会的落伍",
           "太平天国运动的历史意义与局限",
           "洋务运动的历史意义与局限",
           "辛亥革命的纲领，意义是什么？",
           "新文化运动的内容是",
           "毛泽东思想形成的标志是什么？",
           "中国共产党史上生死攸关的转折点是什么？",
           "中国抗日战争在世界反法西斯战争中的地位和作用？",
           "中国共产党领导的多党合作，政治协商的格局是怎样形成的？",
            "中国革命胜利的原因和基本经验有哪些？",
            "新民主主义社会的性质",
            "中国实行社会主义改造的国内外条件？",
            "新冠肺炎疫情发生后，党中央将疫情防控作为头等大事来抓，习近平总书记亲自指挥、亲自部署，坚持把什么放在第一位？",
            "深化供给侧结构性改革，必须持续推进(   )，优化市场供求结构",
            "近代以来中华民族最伟大的梦想是什么？",
            "习近平总书记强调，我们党全面领导、长期执政，面临的最大挑战是什么？",
            "习近平强军思想是什么",
            "党的十九届四中全会指出，党和人民在长期实践探索中形成的科学制度体系是指",
            "2020年脱贫攻坚任务完成后，我国将有1亿左右贫困人口实现脱贫，这也意味着我国将提前（）实现联合国2030年可持续发展议程的减贫目标",
            "党的十九大提出：从2035年到本世纪中叶，在基本实现现代化的基础上，再奋斗15年，把我国建成（）的社会主义现代化强国",
            "新时代党的建设总要求是坚持和加强党的全面领导，坚持党要管党、全面从严治党，以(   )为主线",
            "习近平总书记在参加内蒙古代表团的审议时发表重要讲话指出，我们党没有自己特殊的利益，党在任何时候都把（）放在第一位",
            "2014年十八届四中全会通过了(   )，明确提出全面推进依法治国，加快建设法治中国，开启了中国特色社会主义法治道路的新征程",
            "中国共产党 ( ) 将毛泽东思想规定为中国共产党的一切工作的指针"
          ]
for problem in problems:
    res = vectordb.similarity_search_with_relevance_scores(query=problem, k=1)
    print(res)
    print("-----------")

[(Document(metadata={'page': 1, 'source': './《习近平谈治国理政》第二卷.pdf'}, page_content='人，顺应时代发展，从理论和实践结合上系统回答了新时代坚持和\n发展什么样的中国特色社会主义、怎样坚持和发展中国特色社会主\n义这个重大时代课题，创立了习近平新时代中国特色社会主义思\n想，为决胜全面建成小康社会、夺取新时代中国特色社会主义伟大\n胜利、实现中华民族伟大复兴的中国梦、实现人民对美好生活的向\n往提供了行动指南，也为推动构建人类命运共同体、促进人类和平\n与发展事业贡献了中国智慧和中国方案。中共十九大把习近平新时\n代中国特色社会主义思想确立为中国共产党必须长期坚持的指导思\n想，实现了中国共产党的指导思想又一次与时俱进。\n习近平是新时代中国特色社会主义思想的主要创立者。2014年'), 0.4038293717718261)]
-----------
[(Document(metadata={'page': 53, 'source': './《习近平谈治国理政》第四卷.pdf'}, page_content='当注意的。”近代以后，中国社会矛盾错综复杂，有帝国主义和\n中华民族的矛盾、封建主义和人民大众的矛盾，有资产阶级和无\n产阶级的矛盾、反动统治阶级内部的矛盾，而帝国主义和中华民'), 0.541105956350356)]
-----------
[(Document(metadata={'page': 55, 'source': './《习近平谈治国理政》第一卷.pdf'}, page_content='习近平谈治国理政\n党的十八大精神，说一千道一万，归结为一点，就是坚持\n和发展中国特色社会主义。今年是邓小平同志提出建设中国特\n色社会主义进入 31个年头了。邓小平同志开创了中国特色社\n会主义，第一次比较系统地初步回答了在中国这样经济文化比\n较落后的国家如何建设社会主义、如何巩固和发展社会主义的\n·系列基本问题，用新的思想观点，继承和发展了马克思主义，\n开拓了马克思主义新境界，把对社会主义的认识提高到新的科\n学水平。\n中国特色社会主义是社会主义而不是其他什么主义，科学\n社会主义基本原则不能丢，丢了就不是社会主义。一个国家实\n行什么样的主义，关键要看这个主义能否解决这

In [43]:
snapshot_download('maidalun/bce-embedding-base_v1', cache_dir='./')

2024-09-23 15:53:51,842 - modelscope - WARNING - Model revision not specified, use revision: v0.0.3


'./maidalun/bce-embedding-base_v1'

In [44]:
bce_embedding_model_path = "./maidalun/bce-embedding-base_v1"

bce = get_embedding(bce_embedding_model_path)

loaders_chinese = [
    # PyMuPDFLoader("./data/24徐涛《核心考案》高清无水印PDF【公众号：薄荷考研】.pdf")
        PyPDFLoader("./中国近现代史纲要：2023 年版(1).pdf"),
        PyPDFLoader("./《习近平谈治国理政》第一卷.pdf"),
        PyPDFLoader("./《习近平谈治国理政》第二卷.pdf"),
        PyPDFLoader("./《习近平谈治国理政》第三卷.pdf"),
        PyPDFLoader("./《习近平谈治国理政》第四卷.pdf")
    # 如果需要还可以加入其他文件
]
docs = []
for loader in loaders_chinese:
    pages = loader.load()
    print(len(pages))
    docs.extend(pages)
CHUNK_SIZE = 300
# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)
split_docs = text_splitter.split_documents(docs)
persist_directory = "./vector_db/youdao_bce"

vectordb = Chroma.from_documents(
    documents=split_docs,
    embedding=bce,
    persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
)
vectordb.persist()

429
536
542
427
538


In [45]:
problems = ["近代中国的社会性质是什么？",
           "近代中国的主要矛盾是什么？",
           "如何认识近代中国社会的落伍",
           "太平天国运动的历史意义与局限",
           "洋务运动的历史意义与局限",
           "辛亥革命的纲领，意义是什么？",
           "新文化运动的内容是",
           "毛泽东思想形成的标志是什么？",
           "中国共产党史上生死攸关的转折点是什么？",
           "中国抗日战争在世界反法西斯战争中的地位和作用？",
           "中国共产党领导的多党合作，政治协商的格局是怎样形成的？",
            "中国革命胜利的原因和基本经验有哪些？",
            "新民主主义社会的性质",
            "中国实行社会主义改造的国内外条件？",
            "新冠肺炎疫情发生后，党中央将疫情防控作为头等大事来抓，习近平总书记亲自指挥、亲自部署，坚持把什么放在第一位？",
            "深化供给侧结构性改革，必须持续推进(   )，优化市场供求结构",
            "近代以来中华民族最伟大的梦想是什么？",
            "习近平总书记强调，我们党全面领导、长期执政，面临的最大挑战是什么？",
            "习近平强军思想是什么",
            "党的十九届四中全会指出，党和人民在长期实践探索中形成的科学制度体系是指",
            "2020年脱贫攻坚任务完成后，我国将有1亿左右贫困人口实现脱贫，这也意味着我国将提前（）实现联合国2030年可持续发展议程的减贫目标",
            "党的十九大提出：从2035年到本世纪中叶，在基本实现现代化的基础上，再奋斗15年，把我国建成（）的社会主义现代化强国",
            "新时代党的建设总要求是坚持和加强党的全面领导，坚持党要管党、全面从严治党，以(   )为主线",
            "习近平总书记在参加内蒙古代表团的审议时发表重要讲话指出，我们党没有自己特殊的利益，党在任何时候都把（）放在第一位",
            "2014年十八届四中全会通过了(   )，明确提出全面推进依法治国，加快建设法治中国，开启了中国特色社会主义法治道路的新征程",
            "中国共产党 ( ) 将毛泽东思想规定为中国共产党的一切工作的指针"
          ]
for problem in problems:
    res = vectordb.similarity_search_with_relevance_scores(query=problem, k=1)
    print(res)
    print("-----------")

[(Document(metadata={'page': 33, 'source': './中国近现代史纲要：2023 年版(1).pdf'}, page_content='从近代中国的历史进程，可以看到中国半殖民地半封建社会有以下一\n些基本特征：\n第一，资本－帝国主义侵略势力不但逐步操纵了中国的财政和经济命\n脉，而且逐步控制了中国的政治，日益成为支配中国的决定性力狱。\n第二，中国的封建势力日益衰败并同外国侵略势力勾结，成为资本－\n帝国主义压迫、奴役中国人民的社会基础和统治支柱。\n第三，中国自然经济的基础虽然遭到破坏，但是封建剥削制度的根基\n即封建地主的土地所有制依然在广大地区内保持着，成为中国发展进步的\n严重障碍。\n第四，中国新兴的民族资本主义经济虽然已经产生，并在政治、文化\n生活中起了一定的作用，但是在帝国主义和封建主义的压迫下，它的发展'), 0.5449518984238236)]
-----------
[(Document(metadata={'page': 36, 'source': './中国近现代史纲要：2023 年版(1).pdf'}, page_content='动派。这时，外国侵略者和国内封建统治者完全公开站在一条战线上。例\n如，太平天国后期，清政府向外国侵略者“借师助剿”，共同镇压太平天\n同农民起义，就属于这种情况。\n中国近代社会的主要矛盾决定了，为了使中国在世界上站起来，为了\n使中国入民过上幸福、富裕的生活，就必须推翻帝国主义、封建主义联合\n统治的半殖民地半封建的社会制度，争得民族独立和人民解放；就必须改\n变中国经济技术落后的面貌，实现们家宫强和人民幸福。这是近代以来中\n华民族面临的两大历史任务。无数的志士仁人，一代又一代的中国人，正\n是为此而进行了不屈不挠、英勇顽强的斗争。\n人心” l”IJi夕时i止对中『司的侵略\n｀、，矿巾伈略'), 0.571559203888751)]
-----------
[(Document(metadata={'page': 33, 'source': './中国近现代史纲要：2023 年版(1).pdf'}, page_content='从近代中国的历史进程，可以看到中国半殖民地半封建社会有以下一\n些基本特征：\n第一，资本－帝国主义侵略势力不但逐步操纵了中国的

In [46]:
snapshot_download('iic/nlp_corom_sentence-embedding_chinese-base', cache_dir='./')

2024-09-23 16:05:50,107 - modelscope - WARNING - Model revision not specified, use revision: v1.1.0


'./iic/nlp_corom_sentence-embedding_chinese-base'

In [47]:
corom_embedding_model_path = "./iic/nlp_corom_sentence-embedding_chinese-base"

corom = get_embedding(corom_embedding_model_path)

loaders_chinese = [
    # PyMuPDFLoader("./data/24徐涛《核心考案》高清无水印PDF【公众号：薄荷考研】.pdf")
        PyPDFLoader("./中国近现代史纲要：2023 年版(1).pdf"),
        PyPDFLoader("./《习近平谈治国理政》第一卷.pdf"),
        PyPDFLoader("./《习近平谈治国理政》第二卷.pdf"),
        PyPDFLoader("./《习近平谈治国理政》第三卷.pdf"),
        PyPDFLoader("./《习近平谈治国理政》第四卷.pdf")
    # 如果需要还可以加入其他文件
]
docs = []
for loader in loaders_chinese:
    pages = loader.load()
    print(len(pages))
    docs.extend(pages)
CHUNK_SIZE = 300
# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)
split_docs = text_splitter.split_documents(docs)
persist_directory = "./vector_db/corom"

vectordb = Chroma.from_documents(
    documents=split_docs,
    embedding=corom,
    persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
)
vectordb.persist()

No sentence-transformers model found with name ./iic/nlp_corom_sentence-embedding_chinese-base. Creating a new one with mean pooling.
Some weights of BertModel were not initialized from the model checkpoint at ./iic/nlp_corom_sentence-embedding_chinese-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


429
536
542
427
538


In [48]:
problems = ["近代中国的社会性质是什么？",
           "近代中国的主要矛盾是什么？",
           "如何认识近代中国社会的落伍",
           "太平天国运动的历史意义与局限",
           "洋务运动的历史意义与局限",
           "辛亥革命的纲领，意义是什么？",
           "新文化运动的内容是",
           "毛泽东思想形成的标志是什么？",
           "中国共产党史上生死攸关的转折点是什么？",
           "中国抗日战争在世界反法西斯战争中的地位和作用？",
           "中国共产党领导的多党合作，政治协商的格局是怎样形成的？",
            "中国革命胜利的原因和基本经验有哪些？",
            "新民主主义社会的性质",
            "中国实行社会主义改造的国内外条件？",
            "新冠肺炎疫情发生后，党中央将疫情防控作为头等大事来抓，习近平总书记亲自指挥、亲自部署，坚持把什么放在第一位？",
            "深化供给侧结构性改革，必须持续推进(   )，优化市场供求结构",
            "近代以来中华民族最伟大的梦想是什么？",
            "习近平总书记强调，我们党全面领导、长期执政，面临的最大挑战是什么？",
            "习近平强军思想是什么",
            "党的十九届四中全会指出，党和人民在长期实践探索中形成的科学制度体系是指",
            "2020年脱贫攻坚任务完成后，我国将有1亿左右贫困人口实现脱贫，这也意味着我国将提前（）实现联合国2030年可持续发展议程的减贫目标",
            "党的十九大提出：从2035年到本世纪中叶，在基本实现现代化的基础上，再奋斗15年，把我国建成（）的社会主义现代化强国",
            "新时代党的建设总要求是坚持和加强党的全面领导，坚持党要管党、全面从严治党，以(   )为主线",
            "习近平总书记在参加内蒙古代表团的审议时发表重要讲话指出，我们党没有自己特殊的利益，党在任何时候都把（）放在第一位",
            "2014年十八届四中全会通过了(   )，明确提出全面推进依法治国，加快建设法治中国，开启了中国特色社会主义法治道路的新征程",
            "中国共产党 ( ) 将毛泽东思想规定为中国共产党的一切工作的指针"
          ]
for problem in problems:
    res = vectordb.similarity_search_with_relevance_scores(query=problem, k=1)
    print(res)
    print("-----------")

[(Document(metadata={'page': 16, 'source': './中国近现代史纲要：2023 年版(1).pdf'}, page_content='几千年的君主专制制度，但未能改变中国半殖民地半封建的社会性质和中\n心《毛泽东文集》第八卷，人民出版社 1999年版，笫 340页。'), 0.5442284477691341)]
-----------
[(Document(metadata={'page': 53, 'source': './《习近平谈治国理政》第四卷.pdf'}, page_content='当注意的。”近代以后，中国社会矛盾错综复杂，有帝国主义和\n中华民族的矛盾、封建主义和人民大众的矛盾，有资产阶级和无\n产阶级的矛盾、反动统治阶级内部的矛盾，而帝国主义和中华民'), 0.6052762612289811)]
-----------
[(Document(metadata={'page': 115, 'source': './中国近现代史纲要：2023 年版(1).pdf'}, page_content="102 '\\, !'I I'! i ! i i，勹，丿 V,1,I.I I I I, l'! i, i, 1 1,·, (!' \n中国先进分子选择了马克思主义，这是具有伟大历史意义的事件。\n毛汗东指出：“自从中旧人学会了马克思列宁上义以丿五，中国人 h精神 L\n就由被动转入主动。从这时起，近代世界历史上那种看不起中国人，看不\n起中国文化的时代应甘完结了。 ”(i)\n` 1:i兄息I文勹中 1三J`ll入运动旧芞合\n中门只广见的早期组织 随着中国上人阶级开始作为独立的政治力忙\n登上历史舞台和马克思主义在中国的逐步传播，建立一个以马克思主义为\n指导的丁人阶级政党的任务被提上了日程。"), 0.44373388150139503)]
-----------
[(Document(metadata={'page': 6, 'source': './中国近现代史纲要：2023 年版(1).pdf'}, page_content='一、抵御外来侵略的斗争历程／35\n二、义和团运动与列强瓜分中国图谋咐砐产／38\n反侵略战争的失败与民族意识的觉醒／41\n一、反侵略战争的失败及其原因／41\n二、民族意识的觉醒